In [23]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

from soilgasflux_fcs import models, json_reader
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
nc_files = []
nc_path = pathlib.Path(r'/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/results')
for file in nc_path.glob('*8k*bestPareto.nc'):
    print(file.name)
    nc_files.append(file)

raw_path = pathlib.Path(r'/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data')

test8kMC_20250822_bestPareto.nc
test8kMC_20250826_bestPareto.nc
test8kMC_20250825_bestPareto.nc
test8kMC_20250828_bestPareto.nc
test8kMC_20250829_bestPareto.nc
test8kMC_20250824_bestPareto.nc
test8kMC_20250827_bestPareto.nc
test8kMC_20250823_bestPareto.nc


In [60]:
for nc_file in nc_files:
    ds = xr.open_dataset(nc_file)
    date = pd.to_datetime(ds.coords['time'].values[0]).date()
    selected_raw_folder = raw_path / date.strftime('%Y-%m-%d')
    print(selected_raw_folder)

    a = json_reader.Initializer(folderPath=selected_raw_folder)
    df = a.prepare_rawdata()

    # df['e_s'] = models.calculate_saturated_vapor_pressure(temperature=df['si_temperature'].values)
    df['X_h2o'] = models.mole_fraction_water_vapor(temperature=df['si_temperature'].values,
                                                   humidity=df['si_humidity'].values,
                                                   pressure=df['bmp_pressure'].values/1000)
    df_groupby = df.groupby(by='id').head(1)
    # print(df.groupby(by='id').head(1))
    # print(ds['dcdt(HM)'])
    sgfs = []
    for time in ds['time'].values:
        # print(ds.sel(time=time)['dcdt(HM)'].values)
        # print(df_groupby['si_temperature'].values)
        dcdt = ds.sel(time=time)['dcdt(HM)'].values
        df_groupby_selected = df_groupby[df_groupby['datetime'] == pd.to_datetime(time)]
        sgf = models.soilgasflux(volume=6283, area=314,
                                 p0=df_groupby_selected['bmp_pressure'].values/1000,
                                 w0=df_groupby_selected['X_h2o'].values,
                                 t0=df_groupby_selected['si_temperature'].values,
                                 dcdt=dcdt)
        sgfs.append(sgf)
        # ds.sel(time=time)['sgf'] = sgf
        # print(sgf)
        # break
    ds['sgf'] = (('time','MC'), np.array(sgfs))
    ds.to_netcdf(nc_file.with_name(nc_file.stem + '_with_sgf.nc'))
    # print(ds)
    # break
    # for 

/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-22
/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-26


/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/json_reader.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime_utc'])


ValueError: operands could not be broadcast together with shapes (0,) (8000,) 

In [61]:
for nc_file in nc_files:
    ds = xr.open_dataset(nc_file)
    date = pd.to_datetime(ds.coords['time'].values[0]).date()
    selected_raw_folder = raw_path / date.strftime('%Y-%m-%d')
    print(selected_raw_folder)

    a = json_reader.Initializer(folderPath=selected_raw_folder)
    df = a.prepare_rawdata()

    # df['e_s'] = models.calculate_saturated_vapor_pressure(temperature=df['si_temperature'].values)
    df['X_h2o'] = models.mole_fraction_water_vapor(temperature=df['si_temperature'].values,
                                                   humidity=df['si_humidity'].values,
                                                   pressure=df['bmp_pressure'].values/1000)
    df_groupby = df.groupby(by='id').head(1)
    
    # Collect data arrays for all variables
    sgfs = []
    pressures = []
    temperatures = []
    humidities = []
    x_h2os = []
    
    for time in ds['time'].values:
        dcdt = ds.sel(time=time)['dcdt(HM)'].values
        df_groupby_selected = df_groupby[df_groupby['datetime'] == pd.to_datetime(time)]
        
        if len(df_groupby_selected) > 0:
            pressure = df_groupby_selected['bmp_pressure'].values[0]/1000
            temperature = df_groupby_selected['si_temperature'].values[0]
            humidity = df_groupby_selected['si_humidity'].values[0]
            x_h2o = df_groupby_selected['X_h2o'].values[0]
            
            sgf = models.soilgasflux(volume=6283, area=314,
                                     p0=pressure,
                                     w0=x_h2o,
                                     t0=temperature,
                                     dcdt=dcdt)
        else:
            pressure = np.nan
            temperature = np.nan
            humidity = np.nan
            x_h2o = np.nan
            sgf = np.full_like(dcdt, np.nan)
        
        sgfs.append(sgf)
        pressures.append(pressure)
        temperatures.append(temperature)
        humidities.append(humidity)
        x_h2os.append(x_h2o)
    
    # Add all variables to the dataset
    ds['sgf'] = (('time','MC'), np.array(sgfs))
    ds['pressure'] = (('time',), pressures)
    ds['temperature'] = (('time',), temperatures)
    ds['humidity'] = (('time',), humidities)
    ds['X_h2o'] = (('time',), x_h2os)
    
    ds.to_netcdf(nc_file.with_name(nc_file.stem + '_with_sgf.nc'))
    # print(ds)
    # break
    # for 

/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-22
/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-26


/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/json_reader.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime_utc'])


/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-25
/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-28
/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-29
/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-24
/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-27


/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/json_reader.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime_utc'])


/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-23
